# RAG Indexing - Specialized Embeddings

## 1. Initialization

In [1]:
!nvidia-smi

Wed Jun  5 16:43:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install langchain==0.1.15
!pip install pymilvus==2.4.0
!pip install sentence-transformers==2.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.5/814.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.7/189.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install grpcio==1.56.0 #RUNTIME NEEDS RESTART

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 16.4 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.60.0
    Uninstalling grpcio-1.60.0:
      Successfully uninstalled grpcio-1.60.0


In [5]:
! pip install -U ragatouille

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.0 MB/s e

In [6]:
!pip install tensorflow-text==2.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 18.3 MB/s eta 0:00:00


In [1]:
import json
from transformers import AutoTokenizer
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_community.llms import HuggingFaceEndpoint
from tqdm import tqdm
from ragatouille import RAGPretrainedModel
import json

from getpass import getpass
from typing import List, Optional
from datetime import datetime
from tqdm import tqdm

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from scipy.special import softmax
import json



RAGATOUILLE is developed by Meta AI and enhances traditional NLP models by integrating retrieval mechanisms with generative capabilities. This approach allows the model to fetch relevant documents or information from a large corpus to improve the quality and accuracy of generated responses. By combining retrieval and generation, RAGATOUILLE aims to produce more informative, contextually rich, and accurate outputs, pushing the boundaries of what language models can achieve.

In [2]:
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [3]:
# Function that read a json
def read_json(file_path):
    """
    Parameters:
    - file_path (str): Path to the JSON file.

    Returns:
    - data: Parsed JSON data.
    """
    with open(file_path, 'r', encoding='utf8') as file:
        data = json.load(file)
    return data

In [4]:
# Path to the file containing all the documents
docs_dir = "/content/drive/MyDrive/dataset/documents.json"

In [5]:
# Saving the data in a variable
data=read_json(docs_dir)
print(data[0])

{'id': '0025577043f5090cd603c6aea60f26e236195594', 'kind': 'movie', 'text': '<html><title>Pump Up The Volume Transcript</title><pre>\nHappy Harry Hardon - Did you ever get the feeling that everything in America is \ncompletely fucked up. You know that feeling that the whole country is like one inch \naway from saying \'That\'s it, forget it.\' You think about it. Everything is polluted. The \nenvironment, the government, the schools you name it. Speaking of schools. I was \nwalking the households the other day and I asked myself. Is there live after high school? \nBecause I can\'t face tomorrow, let alone a whole year of this shit. Yeah, you got it folks. \nIt\'s me again with a little attitude for all you out here and waiting for Atlanta. All you \nnice people living in the middle of America the beautiful. Lets see, we\'re on er 92 FM \ntonight and it feels like a nice clean little band so far. No one else is using it. The price is \nright. Heh, heh. And yes folks you guest it. Tonigh

In [6]:
# Printing the amount of questions given as reference
dirDocDomande="/content/drive/MyDrive/dataset/subsampled_golden_pairs.json"
dataDomande=read_json(dirDocDomande)
ids=set()
for elemento in dataDomande:
  ids.add(elemento['id'])
print(len(ids))

83


In [7]:
# Saving the link between titles and ids in a file
title_to_id=dict()
i=0
for id in ids:
  title_to_id[id]=i
  i+=1
with open('/content/drive/MyDrive/title_to_id.json', 'w') as fp:
    json.dump(title_to_id, fp)

## 2. RAG Indexing using ColBERT

ColBERT (Contextualized Late Interaction over BERT) is an advanced retrieval method in natural language processing that leverages the BERT model to improve both efficiency and accuracy in large-scale information retrieval tasks. Unlike traditional cross-encoders that evaluate every possible pair of query and document at query time, ColBERT precomputes document embeddings and interacts with query embeddings in a late stage. This late-stage interaction enables faster and more scalable retrieval while maintaining high accuracy, making it particularly suitable for applications requiring quick access to large datasets.


In [8]:
# Organizing the structures to pass to the ColBERT model
my_documents = []
document_ids = []
document_metadatas = []

selected_titles=[]
for elemento in dataDomande:
  selected_titles.append(elemento['id'])

documentiFiltrati=[]
docInseriti=[]
num_docs = 0
count_filtered_titles = 0

for elemento in data:
  if elemento['id'] in selected_titles:
    documentiFiltrati.append(elemento)
for elemento in documentiFiltrati:
  num_docs+=1
  id = elemento['id']
  if id in docInseriti:
    continue
  docInseriti.append(id)
  sezioneTesto = elemento['text']
  sezioneSummary = elemento['summary']
  text = sezioneTesto + sezioneSummary
  sourceId = title_to_id[id]
  if(sourceId == ""):
    print("hi")
  sourceAlphanum = id

  my_documents.append(text)
  document_ids.append(sourceAlphanum)
  document_metadatas.append({"source_id: " : sourceId})

In [9]:
# RAG implementation
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

index_path = RAG.index(
    index_name="CollectionSaccomannoBertolinoFerrariColBERT2",
    collection=my_documents,
    document_ids=document_ids,
    document_metadatas=document_metadatas,
)

---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Jun 05, 16:50:36] #> Creating directory .ragatouille/colbert/indexes/CollectionSaccomannoBertolinoFerrariColBERT2 


[Jun 05, 16:50:37] [0] 		 #> Encoding 32888 passages..
[Jun 05, 16:52:52] [0] 		 avg_doclen_est = 169.0104522705078 	 len(local_sample) = 32,888
[Jun 05, 16:53:03] [0] 		 Creating 32,768 partitions.
[Jun 05, 16:53:03] [0] 		 *Estimated* 5,950,350 embeddings.
[Jun 05, 16:53:03] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/CollectionSaccomannoBertolinoFerrariColBERT2/plan.json ..
used 20 iterations (5.7298s

/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


[Jun 05, 16:54:51] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


[0.046, 0.045, 0.047, 0.043, 0.045, 0.044, 0.043, 0.043, 0.044, 0.045, 0.042, 0.044, 0.044, 0.045, 0.045, 0.047, 0.04, 0.043, 0.043, 0.042, 0.043, 0.045, 0.044, 0.044, 0.044, 0.042, 0.044, 0.044, 0.045, 0.045, 0.044, 0.047, 0.049, 0.044, 0.044, 0.041, 0.043, 0.042, 0.043, 0.052, 0.047, 0.043, 0.045, 0.043, 0.046, 0.042, 0.042, 0.048, 0.047, 0.042, 0.043, 0.043, 0.044, 0.045, 0.044, 0.043, 0.048, 0.046, 0.051, 0.043, 0.041, 0.046, 0.045, 0.047, 0.047, 0.045, 0.047, 0.044, 0.039, 0.042, 0.046, 0.043, 0.042, 0.044, 0.043, 0.043, 0.046, 0.052, 0.044, 0.045, 0.046, 0.045, 0.044, 0.044, 0.04, 0.042, 0.044, 0.043, 0.042, 0.055, 0.043, 0.047, 0.043, 0.048, 0.042, 0.044, 0.048, 0.04, 0.045, 0.044, 0.046, 0.044, 0.044, 0.049, 0.045, 0.041, 0.045, 0.042, 0.044, 0.042, 0.044, 0.044, 0.046, 0.042, 0.044, 0.045, 0.046, 0.046, 0.047, 0.043, 0.043, 0.042, 0.043, 0.048, 0.043, 0.043, 0.043, 0.043]


...

[Jun 05, 16:58:14] [0] 		 #> Encoding 10207 passages..


2it [02:31, 75.64s/it]
100%|██████████| 2/2 [00:00<00:00, 114.35it/s]


[Jun 05, 16:58:59] #> Optimizing IVF to store map from centroids to list of pids..
[Jun 05, 16:58:59] #> Building the emb2pid mapping..
[Jun 05, 16:58:59] len(emb2pid) = 5950312


100%|██████████| 32768/32768 [00:00<00:00, 48333.73it/s]


[Jun 05, 16:59:00] #> Saved optimized IVF to .ragatouille/colbert/indexes/CollectionSaccomannoBertolinoFerrariColBERT2/ivf.pid.pt
Done indexing!


In [10]:
# Query example
query = "Why did Reverend Mother chastise Deloris?"
RAG = RAGPretrainedModel.from_index(index_path)
results = RAG.search(query)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading searcher for index CollectionSaccomannoBertolinoFerrariColBERT2 for the first time... This may take a few seconds
[Jun 05, 16:59:04] #> Loading codec...
[Jun 05, 16:59:04] #> Loading IVF...
[Jun 05, 16:59:04] #> Loading doclens...


100%|██████████| 2/2 [00:00<00:00, 517.40it/s]

[Jun 05, 16:59:04] #> Loading codes and residuals...



100%|██████████| 2/2 [00:00<00:00,  7.59it/s]


Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . Why did Reverend Mother chastise Deloris?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2339,  2106, 10547,  2388, 15775, 29472,  2063,  3972,
        21239,  1029,   102,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')



In [11]:
# Output of the example
results

[{'content': 'Mary Clarence objects to following the strictures and simple life of the convent, but comes to befriend several of the nuns, including the forever jolly Sister Mary Patrick, quiet and meek Sister Mary Robert, and the elderly deadpan Sister Mary Lazarus. After sneaking into a nearby bar, Mary Clarence is chastised by Reverend Mother and put into the choir, which she has seen to be dreadful. The choir nuns, learning that Mary Clarence has a background in music, elect her to take over as choir director, which she accepts, and she rearranges them to make them better singers. At Mass one Sunday, the choir sings the "Hail Holy Queen" in the traditional manner beautifully before shifting into a gospel and rock-and-roll-infused performance of the hymn.\nReverend Mother is infuriated with Mary Clarence about the performance, and orders that Mary Lazarus once again become the leader the choir, but Monsignor O\'Hara is thrilled with the performance as the unorthodox music brought pe

## 3. RAG Retrieval

In [12]:
# Retrieval model setup
CHUNK_SIZE = 8192
CHUNK_OVERLAP = 0
SIMILARITY_MEASURE = "COSINE"
K = 10
EMBEDDINGS_MODEL = "colbert-ir/colbertv2.0"

# Generation model setup
LLM_MODEL_NAME = "mistralai/Mixtral-8x7B-Instruct-v0.1"
LLM_MAX_TOKENS = 128000
RESPONSE_MAX_TOKENS = 100

In [13]:
# Access to HuggingFace
HUGGINGFACEHUB_API_TOKEN = getpass()

import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

··········


In [14]:
llm = HuggingFaceEndpoint(repo_id=LLM_MODEL_NAME, temperature=0.01, max_new_tokens=RESPONSE_MAX_TOKENS)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [15]:
qa_chain = load_qa_chain(llm, chain_type = 'stuff', verbose=False)

In [16]:
# Functions to standardize the text
def get_prompt(
    chain: StuffDocumentsChain,
    context: str,
    is_question_answering_action: bool,
    human_input: Optional[str],
) -> str:
    inputs = {}
    if is_question_answering_action:
        inputs = {
            'question': human_input,
            'context': context
        }
    else:
        inputs = {
            'text': context
        }
    prompt = chain.llm_chain.prompt.format(**inputs)
    return prompt


def adapt_to_input_size(
    context_messages_texts: List[str],
    model: str,
    max_input_tokens: int,
    max_new_tokens_in_the_answer: int,
    chain: StuffDocumentsChain,
    is_question_answering_action: bool,
    human_input: Optional[str],
    verbose: bool
) -> List[str]:
    """100 tokens ~= 75 words Source: https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them"""
    context = chain.document_separator.join(context_messages_texts)
    prompt = get_prompt(chain, context, is_question_answering_action, human_input)
    input_size = chain.llm_chain.llm.get_num_tokens(prompt)
    # input_size: totale numero tokens che arriveranno all'LLM
    max_input_size_for_request = max_input_tokens - max_new_tokens_in_the_answer
    if input_size >= max_input_size_for_request:
        enc = AutoTokenizer.from_pretrained(model)
        #print(f"The number of input tokens {input_size} exceed the maximum number allowed for the request: {max_input_size_for_request}")
        overflow_tokens = input_size - max_input_size_for_request
        len_context_encoding = chain.llm_chain.llm.get_num_tokens(context)
        slice_size = len_context_encoding - overflow_tokens - len(context_messages_texts) + 1 # Last two terms are used to exclude separators from the count
        #print(f"Currently the context is long {len_context_encoding - len(context_messages_texts) + 1} tokens, will be pruned to the first {slice_size} tokens")
        current_tokens = 0
        result = []
        for context_text in context_messages_texts:
            page_tokenized = enc.encode(context_text)
            if current_tokens + len(page_tokenized) <= slice_size:
                current_tokens += len(page_tokenized)
                result.append(context_text)
            else:
                page_tokenized_slice = page_tokenized[:(slice_size - current_tokens)]
                result.append(enc.decode(page_tokenized_slice))
                current_tokens += len(page_tokenized_slice)
                assert current_tokens == slice_size
                break
        # here the context is correctly pruned
    else:
        result = context_messages_texts

    if verbose:
        new_context = chain.document_separator.join(result)
        new_prompt = get_prompt(chain, new_context, is_question_answering_action, human_input)
        new_input_size = chain.llm_chain.llm.get_num_tokens(new_prompt)
        #print(f'A total of {new_input_size} tokens will be spent')
    return result

In [17]:
# Uploading queries documents
ANNOTATIONS_PATH = "/content/drive/MyDrive/dataset/subsampled_golden_pairs.json"
TITLE_TO_ID_PATH = "/content/drive/MyDrive/title_to_id.json"
with open(ANNOTATIONS_PATH, 'r') as json_file:
    anns = json.load(json_file)
with open(TITLE_TO_ID_PATH, 'r') as json_file:
    title_to_id = json.load(json_file)
queries=dict()
for elemento in anns:
  id=elemento['id']
  domanda=elemento['question']
  risposte=elemento['answers']
  queries[domanda]=(id,title_to_id[id])

In [18]:
queries.items()

dict_items([('Why did Reverend Mother chastise Deloris?', ('f7bb9eb9306b79cad4b6466f2ac3dcbd0e5fa63a', 46)), ('What does Augustus reveal to Hazel after staying in Amsterdam together?', ('2453d062843edc379bdae3be69859e18bf1abd9d', 50)), ('What happen to the hideout spot?', ('08ac6c0dca91fee93d27c7caf85bb57e17d4fc46', 56)), ('How much did Christian pay for the video?', ('6ed5d860df4edd3d32f1aca52ec369f0d21039a0', 57)), ('Where is the underground shelter where James Cole is being held as a prisoner located?', ('5df7b23024d86a042fcdcb06ee485d528317a156', 52)), ('What was the top secret project that the CIA officers watched in the begining?', ('54e7219964513efc449b07327b717c600c609115', 22)), ('In which mountains are Hal and Sarah stranded?', ('34baa9e1ca1e15bccce2326e4f5c52bb6ba30eb8', 54)), ('Why does Daisy tell Benjamin to stay out of his life?', ('21813f2122fca16d05e89d44f4521f7da8a8f3b7', 9)), ('What type of animal was Mar?', ('dcd3468c3e18822f08ff0607edfb3e9ca06f3ef0', 58)), ("What wa

In [19]:
# Setting standard prompt
PROMPT = """You are a Question Answering system that is rewarded if the response is short, concise and straight to the point, use the following pieces of context to answer the question at the end. If the context doesn't provide the required information simply respond <no answer>.
    Context:
    {context}

    Human question: {human_input}
    """

In [20]:
# Printing 10 question-answer outputs
queries_to_print = 10
#print(K)
num_printed_queries = 0
for query, titles in queries.items():
    #print("Query",query)
    #print("Titles",titles)
    if query == "":
        continue
    if num_printed_queries >= queries_to_print:
        break
    print(f"Query: {query}")
    num_printed_queries += 1
    printed_answer = False
    if printed_answer:
        continue
    else:
        printed_answer = True
    results = RAG.search(query)
    adapted_res_texts = adapt_to_input_size(context_messages_texts=[res["content"] for res in results], model=LLM_MODEL_NAME, max_input_tokens=LLM_MAX_TOKENS, max_new_tokens_in_the_answer=RESPONSE_MAX_TOKENS, chain=qa_chain, is_question_answering_action=True, human_input=query, verbose=True)
    adapted_res_docs = [Document(page_content = adapted_res_text) for adapted_res_text in adapted_res_texts]
    prompt = PromptTemplate(input_variables=["human_input","context"], template=PROMPT)
    answer = qa_chain.run(input_documents = adapted_res_docs, question = query, prompt = prompt)
    print(f"Answer: {answer}")
    print(f"\n")

Query: Why did Reverend Mother chastise Deloris?


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (3296 > 1024). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Answer:  Reverend Mother chastised Deloris for singing a gospel and rock-and-roll-infused performance of the hymn "Hail Holy Queen" during Mass.


Query: What does Augustus reveal to Hazel after staying in Amsterdam together?
Answer:  Augustus reveals to Hazel that his cancer has relapsed, has spread throughout his body and is terminal.


Query: What happen to the hideout spot?
Answer:  The hideout spot was found and brought out to prove that Llanfeare was not his own.


Query: How much did Christian pay for the video?
Answer:  Christian paid $1 million for the snuff film.


Query: Where is the underground shelter where James Cole is being held as a prisoner located?
Answer:  The underground shelter where James Cole is being held as a prisoner is located beneath the ruins of Philadelphia.


Query: What was the top secret project that the CIA officers watched in the begining?
Answer:  The top secret project that the CIA officers watched in the begining was called "Crossbow": a space shu

In [22]:
# Generating the answers for each question
res = {}
answers = {}
for query, titles in tqdm(queries.items()):
    print(titles)
    answers[query]=[]
    res[query]=[]
    print(titles)
    results = RAG.search(query)

    res[query].append({'id':titles[1], 'results':[(res['content'], res['score'], res['document_id']) for res in results]})
    adapted_res_texts = adapt_to_input_size(context_messages_texts=[res["content"] for res in results], model=LLM_MODEL_NAME, max_input_tokens=LLM_MAX_TOKENS, max_new_tokens_in_the_answer=RESPONSE_MAX_TOKENS, chain=qa_chain, is_question_answering_action=True, human_input=query, verbose=True)
    adapted_res_docs = [Document(page_content = adapted_res_text) for adapted_res_text in adapted_res_texts]
    prompt = PromptTemplate(input_variables=["human_input","context"], template=PROMPT)
    answer = qa_chain.run(input_documents = adapted_res_docs, question = query, prompt = prompt)
    prompt_input = {'human_input':query, 'context':qa_chain.document_separator.join(adapted_res_texts)}
    parsed_prompt = prompt.format(**prompt_input)
    answers[query].append({'prompt':parsed_prompt,'doc_id':titles[1],'doc_title':titles[0],'answer':answer})
data = datetime.today().strftime('%Y%m%d')

  0%|          | 0/100 [00:00<?, ?it/s]

('f7bb9eb9306b79cad4b6466f2ac3dcbd0e5fa63a', 46)
('f7bb9eb9306b79cad4b6466f2ac3dcbd0e5fa63a', 46)


  2%|▏         | 2/100 [00:00<00:21,  4.48it/s]

('2453d062843edc379bdae3be69859e18bf1abd9d', 50)
('2453d062843edc379bdae3be69859e18bf1abd9d', 50)
('08ac6c0dca91fee93d27c7caf85bb57e17d4fc46', 56)
('08ac6c0dca91fee93d27c7caf85bb57e17d4fc46', 56)


  4%|▍         | 4/100 [00:00<00:14,  6.60it/s]

...

 98%|█████████▊| 98/100 [02:25<00:02,  1.43s/it]

('17f2e2685467b6eebf01d2a643e31f53ea215e91', 11)
('17f2e2685467b6eebf01d2a643e31f53ea215e91', 11)


 99%|█████████▉| 99/100 [02:26<00:01,  1.24s/it]

('e524c75271fe0e5c4b1fdbc3c2e297cdecf1b113', 16)
('e524c75271fe0e5c4b1fdbc3c2e297cdecf1b113', 16)


100%|██████████| 100/100 [02:27<00:00,  1.48s/it]


In [23]:
# Utility functions
RAG_OUTPUT = ""
ALL_OUTPUT_CSV_FILE = f"/content/BemScore_all_categories.csv"

def read_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        print(f"Error: File not found at path '{file_path}'.")
        return None
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file '{file_path}': {e}")
        return None

def read_jsonl(file_path):
    """
    Read a JSONL file and return a list of dictionaries.

    Args:
    - file_path (str): Path to the JSONL file.

    Returns:
    - list: A list of dictionaries containing the JSON objects from the file.
    """
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}. Skipping line.")
    return data

def write_json_file(file_path, data):
    try:
        with open(file_path, 'w', encoding='utf-8') as file:
            json.dump(data, file, indent=4, ensure_ascii=True)
        print(f"Data successfully written to '{file_path}'.")
    except json.JSONDecodeError as e:
        print(f"Error encoding data as JSON: {e}")
    except Exception as e:
        print(f"Error writing data to '{file_path}': {e}")

def compute_metric_on_candidate_answer(candidate_answer, golden_answers, metric_function):
    sum_scores = 0
    number_of_answers = 0
    for golden_answer in golden_answers:
        current_score = metric_function([candidate_answer], [golden_answer])
        sum_scores += current_score
        number_of_answers += 1
    assert number_of_answers > 0
    return round(sum_scores/number_of_answers, 3)

In [26]:
# More utility functions
#@title Sets up the BERT tokenizer using tf-text.
import tensorflow_text as text

VOCAB_PATH = 'gs://cloud-tpu-checkpoints/bert/keras_bert/uncased_L-12_H-768_A-12/vocab.txt'  #@param {type:"string"}

vocab_table = tf.lookup.StaticVocabularyTable(
        tf.lookup.TextFileInitializer(
            filename=VOCAB_PATH,
            key_dtype=tf.string,
            key_index=tf.lookup.TextFileIndex.WHOLE_LINE,
            value_dtype=tf.int64,
            value_index=tf.lookup.TextFileIndex.LINE_NUMBER
        ),
        num_oov_buckets=1)
cls_id, sep_id = vocab_table.lookup(tf.convert_to_tensor(['[CLS]', '[SEP]']))
tokenizer = text.BertTokenizer(vocab_lookup_table=vocab_table,
                               token_out_type=tf.int64,
                               preserve_unused_token=True,
                               lower_case=True)
def bertify_example(example):
  question = tokenizer.tokenize(example['question']).merge_dims(1, 2)
  reference = tokenizer.tokenize(example['reference']).merge_dims(1, 2)
  candidate = tokenizer.tokenize(example['candidate']).merge_dims(1, 2)

  input_ids, segment_ids = text.combine_segments(
      (candidate, reference, question), cls_id, sep_id)

  return {'input_ids': input_ids.numpy(), 'segment_ids': segment_ids.numpy()}


def pad(a, length=512):
  return np.append(a, np.zeros(length - a.shape[-1], np.int32))


def bertify_examples(examples):
  input_ids = []
  segment_ids = []
  for example in examples:
    example_inputs = bertify_example(example)
    input_ids.append(pad(example_inputs['input_ids']))
    segment_ids.append(pad(example_inputs['segment_ids']))

  return {'input_ids': np.stack(input_ids), 'segment_ids': np.stack(segment_ids)}

In [27]:
bem = hub.load('https://kaggle.com/models/google/bert/frameworks/TensorFlow2/variations/answer-equivalence-bem/versions/1')

In [28]:
def get_bem_score(predictions,references):
    prediction = predictions[0]
    reference = references[0]
    examples = [{
    'question': prediction['query'],
    'reference': reference,
    'candidate': prediction['answer']
    }]
    inputs = bertify_examples(examples)
    # The outputs are raw logits.
    raw_outputs = bem(inputs)

    # They can be transformed into a classification 'probability' like so:
    bem_score = float(softmax(np.squeeze(raw_outputs))[1])
    return bem_score

def list_avg(lst):
    return sum(lst) / len(lst)

def compute_aggregates(metrics_dict):
    aggregate_dict = {}
    for key in metrics_dict.keys():
        aggregate_dict[key] = []
        if key == "Query":
            aggregate_dict[key].extend(['MEAN','MIN','MAX'])
            continue
        if key == "Candidate answer" or key == "First gt answer":
            aggregate_dict[key].extend(['','',''])
            continue
        values = metrics_dict[key]
        mean_value = list_avg(values)
        min_value = min(values)
        max_value = max(values)
        aggregate_dict[key].append(round(mean_value,3))
        aggregate_dict[key].append(min_value)
        aggregate_dict[key].append(max_value)
    return aggregate_dict

In [29]:
# Creating one structure for BEM score computing

query_to_answers = {}
for query in queries.items():

    predicted_answers = [elem[1][0]['answer'] for elem in answers.items() if elem[0]==query[0]]
    assert len(predicted_answers) == 1

    golden_answers = [e  for elem in anns for e in elem['answers'] if elem['question']==query[0]]
    golden_answers = [golden_answers[0]]
    assert len(golden_answers) == 1

    query_to_answers[query[0]] = {'predicted_answers':predicted_answers, 'golden_answers':golden_answers}

In [30]:
# Computing BEM score
res = {"Query":[], "Candidate answer":[], "First gt answer":[], "BEM score":[]}
for query in tqdm(queries):
    res["Query"].append(query)
    prediction_answer = query_to_answers[query]['predicted_answers']
    reference_answers = query_to_answers[query]['golden_answers']
    res['Candidate answer'].append(prediction_answer[0])
    res['First gt answer'].append(reference_answers[0][0])
    pred_bem_score = compute_metric_on_candidate_answer({'query':query,'answer':prediction_answer[0]},reference_answers[0], get_bem_score)
    res['BEM score'].append(round(pred_bem_score, 3))
aggregate_dict = compute_aggregates(res)
for key in res.keys():
    res[key].extend(aggregate_dict[key])

100%|██████████| 100/100 [07:24<00:00,  4.44s/it]


In [31]:
# Computing BEM medium score

bem_scores = res['BEM score']

mean_bem_score = sum(bem_scores) / len(bem_scores)

print(f"BEM score generale: {mean_bem_score}")

BEM score generale: 0.5814854368932042
